In [19]:

lyrics = """
In a galaxy so vast, where the eyeballs roam,
Cars race through the stars, finding their home,
Trains on cosmic tracks, in the silence of space,
Guns fire at the orbiting eyes, in an endless chase.

(Chorus)
*Zoom*, *Bang*, the chaos ignites,
Floating through the cosmos, under endless nights,
Crazy dreams, where visions fly,
Through the eye of a storm, in the space-born sky.

(Verse 2)
Sprawling scenes with metal and light,
Eyeballs gaze in wonder, a surreal sight,
Engines roar with a cosmic tune,
Traveling the dreams of the silver moon.

(Bridge)
Craziness in circles, a neon swirl,
Through the binoculars of this wild new world,
Floating, drifting, with eyes wide and bold,
In the space between whispers, stories unfold.

(Chorus)
*Zoom*, *Bang*, the chaos ignites,
Floating through the cosmos, under endless nights,
Crazy dreams, where visions fly,
Through the eye of a storm, in the space-born sky.

Visual Theme: A surrealist, animated, dreamlike vision where the viewer is plunged into a sprawling cosmic scene. Cars and trains travel along ethereal tracks through vast starry expanses, accompanied by the whimsical yet slightly unsettling sight of eyeballs floating through space. The music video is painted in vibrant, shifting colors that flicker and pulse in tune with the electric beats. Guns announce their presence with flashes of neon as they engage the floating eyes in a strange, chaotic ballet. The theme of the song echoes the chaotic beauty of dreams—unexpected, vivid, and boundless in imagination, inviting the viewer to embrace the wonder of the unconventionally surreal.
"""


In [38]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip
from moviepy.video.fx.loop import loop
import ffmpeg
import asyncio
import time
import os

async def merge_videos_and_song_with_subtitles(srt_path):
    # Step 1: Download song and videos
    # async with ClientSession() as session:
    #     song_path = await download_file(session, song_url, 'song.mp3')
    #     video_paths = []
    #     for i, video_url in enumerate(video_urls):
    #         video_path = await download_file(session, video_url, f'video_{i+1}.mp4')
    #         video_paths.append(video_path)

    song_path = './song.mp3'
    video_paths = [f'./video_{i+1}.mp4' for i in range(2)]

    # Step 2: Load videos using MoviePy
    video_clips = [VideoFileClip(video) for video in video_paths]
    
    # Step 3: Concatenate and loop the videos until they match the song duration
    concatenated_clip = concatenate_videoclips(video_clips)

    song_audio = AudioFileClip(song_path)
    song_duration = song_audio.duration
    final_video_clip = loop(concatenated_clip, duration=song_duration)
    final_video_clip = final_video_clip.set_audio(song_audio)

    # Step 4: Add subtitles using ffmpeg (MoviePy)
    # MoviePy uses ffmpeg to overlay the subtitles without ImageMagick.
    final_video_with_subtitles = final_video_clip.subclip(0, song_duration).set_duration(song_duration).set_audio(song_audio)

    # Step 5: Write the output to a file
    output_path = f"final_output_with_subtitles_{time.time()}.mp4"
    final_video_with_subtitles.write_videofile(output_path, codec='libx265', audio_codec='aac', ffmpeg_params=['-vf', f"subtitles={srt_path}"])

    # Step 6: Clean up temporary files
    song_audio.close()
    final_video_clip.close()
    concatenated_clip.close()
    final_video_with_subtitles.close()
    for video in video_clips:
        video.close()
    
    return output_path

In [39]:
await merge_videos_and_song_with_subtitles('./output.ass')


Moviepy - Building video final_output_with_subtitles_1729430425.2236214.mp4.
MoviePy - Writing audio in final_output_with_subtitles_1729430425.2236214TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output_with_subtitles_1729430425.2236214.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_subtitles_1729430425.2236214.mp4


'final_output_with_subtitles_1729430425.2236214.mp4'

In [16]:
import stable_whisper

In [17]:
model = stable_whisper.load_model("base")

c:\Users\pmazu\Documents\GitHub\BlinkStory\.venv\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

In [20]:
# remove everything in brackets of any sory from lyrics
import re
lyrics = re.sub(r'\[.*?\]|\(.*?\)', '', lyrics)


In [35]:
result = model.align('./song.mp3', lyrics, language='en', fast_mode=True)


Align: 100%|██████████| 169.63/169.63 [00:01<00:00, 122.90sec/s]
Adjustment: 100%|██████████| 147.92/147.92 [00:00<00:00, 49278.91sec/s]


In [36]:
for segment in result:
    print("[%.2f -> %.2f] %s" % (segment.start, segment.end, segment.text))



[12.10 -> 14.26]  In a galaxy so vast,
[14.50 -> 15.88]  where the eyeballs roam,
[18.86 -> 20.96]  Cars race through the stars,
[21.34 -> 23.00]  finding their home,
[26.02 -> 28.14]  Trains on cosmic tracks,
[29.02 -> 31.08]  in the silence of space,
[31.08 -> 35.62]  Guns fire at the orbiting eyes,
[35.62 -> 37.64]  in an endless chase.
[37.64 -> 40.00]    *Zoom*,
[40.00 -> 43.60]  *Bang*, the chaos ignites, Floating through the cosmos,
[43.60 -> 50.28]  under endless nights, Crazy dreams, where visions fly, Through the eye of a storm,
[50.28 -> 53.22]  in the space-born sky.
[59.32 -> 61.70]    Sprawling scenes with metal and light,
[61.70 -> 63.38]  Eyeballs gaze in wonder,
[63.38 -> 68.64]  a surreal sight, Engines roar with a cosmic tune,
[68.64 -> 71.36]  Traveling the dreams of the silver moon.
[71.84 -> 73.08]    Craziness in circles,
[73.52 -> 74.32]  a neon swirl,
[74.62 -> 78.04]  Through the binoculars of this wild new world,
[78.40 -> 80.90]  Floating, drifting, with eye

In [37]:
srt = result.to_ass('output.ass', karaoke=True)


Saved: c:\Users\pmazu\Documents\GitHub\BlinkStory\output.ass
